In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(2, 4)
net(X)

tensor([[0.2838],
        [0.3911]], grad_fn=<AddmmBackward>)

In [2]:
print(net[0].state_dict())
print(net[1].state_dict())
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.4787,  0.1697, -0.0012,  0.2754],
        [ 0.2457, -0.1963,  0.2908,  0.3125],
        [ 0.2315, -0.2993,  0.4903,  0.0297],
        [-0.3596,  0.2029, -0.1186,  0.0563],
        [ 0.4856, -0.4472,  0.0390,  0.3927],
        [ 0.0505, -0.3858,  0.0484, -0.0961],
        [-0.1296, -0.2121,  0.3868,  0.0820],
        [ 0.4600,  0.0982,  0.2593,  0.4749]])), ('bias', tensor([-0.1444, -0.0468,  0.4544, -0.1255, -0.2558, -0.3249,  0.0416, -0.1747]))])
OrderedDict()
OrderedDict([('weight', tensor([[-0.1236,  0.0492,  0.2612, -0.1377,  0.1109, -0.1536,  0.1353, -0.0898]])), ('bias', tensor([0.1925]))])


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.data.numpy())
print(net[2].bias.detach().numpy())

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1925], requires_grad=True)
tensor([0.1925])
[0.19252989]
[0.19252989]


In [4]:
print(net[2].weight.grad)

None


In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([0.1925])

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3475],
        [-0.3475]], grad_fn=<AddmmBackward>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.3144, -0.4534, -0.2912, -0.2847,  0.1189,  0.1130,  0.1737,  0.1349])

In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
print(net)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


(tensor([ 0.0071, -0.0054,  0.0076,  0.0028]), tensor(0.))

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [13]:
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([ 0.2141, -0.5561,  0.6797, -0.0491])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()])
        print("Init", [(name, param.shape) for name, param in m.named_parameters()])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init ('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
Init [('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
Init ('weight', torch.Size([1, 8])) ('bias', torch.Size([1]))
Init [('weight', torch.Size([1, 8])), ('bias', torch.Size([1]))]


tensor([[ 0.0000,  0.0000, -0.0000,  6.6386],
        [-0.0000,  0.0000, -5.8789, -0.0000]], grad_fn=<SliceBackward>)

In [15]:
a = [1,2,3,4]
print(a)
print(*a)

[1, 2, 3, 4]
1 2 3 4


In [16]:
net[0].weight.data[:] +=1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000,  7.6386])

In [17]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)

tensor([[0.0505],
        [0.0568]], grad_fn=<AddmmBackward>)

In [18]:
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
